# k-means algorithm

k-means clustering (also refered as Lloyd's algorithm) is a method of vector quantization that aims to partition `n` observations into `k` clusters in which each observation belongs to the cluster with the nearest mean (cluster centers or cluster centroid), serving as a prototype of the cluster. This results in a partitioning of the data in Voronoi cells.

k-means has many applications, among them:
- customer segmentation according to a certain criterion (demographic, purchasing habit, ...)
- use of Data Mining clustering when mining data to identify similar individuals
- grouping of documents according to their content (Google news grouping documents by theme for instance)

## Description

Given a set of observations $(\textbf{x}_{1}, \textbf{x}_{2}, ..., \textbf{x}_{n})$, where each observation is a *d*-dimensional real vector, k-means clustering aims to partition the $n$ observations into $k (\leq n)$ sets $\textbf{S} = \{S_{1}, S_{2}, ..., S_{k}\}$ so as to minimize the within-cluster sum of sqaures (i.e. variance). Formally, the objective is to find:

$\arg\underset{\textbf{S}}\min \sum_{i=1}^{k} \sum_{\textbf{x} \in S_{i}} \| \textbf{x} - \boldsymbol{\mu}_{i} \|^2 = \arg\underset{\textbf{S}}\min \sum_{i=1}^{k} |S_{i}| \text{Var}S_{i}$

where $\boldsymbol{\mu}_{i}$ is the mean of points in $S_{i}$.

Choosing $k$ is not intuitive, especially for large data sets for which we have few knowledge. A usual method is to run the k-means algorithm with different values for $k$ and choose the one for which the variance does not decrease much if we increment it.
Plotting the variance function of k shows a decrasing curve that looks like an elbow, therefore this method is refered as the Elbow Method.

## Algorithm

Given an initial set of $k$ means $m_{1}^{(1)}, m_{2}^{(1)}, ..., m_{k}^{(1)}$, the algorithm proceeds by alternating between two steps:

- **Assignment step**: assign each observation to the cluster with the nearest mean (that with the least squared Euclidian distance): $S_{i}^{(t)} = \{ x_{p} : \| x_{p} - m_{i}^{(t)}\|^2 \leq \| x_{p} - m_{j}^{(t)}\|^2 \forall j, 1 \leq j \leq k\}$, where each $x_{p}$  is assigned to exactly one $S^{(t)}$, even if it could be assigned to two or more of them.
- **Update step**:  Recalculate means (centroids) for observations assigned to each cluster: $m_{i}^{(t+1)} = \frac{1}{|S_{i}^{(t)}|} \sum_{x_{j} \in S_{i}^{(t)}} x_{j}$.

The algorithm has converged when the assignments no longer change. The algorithm is not guaranteed to find the optimum.

Many methods exist for choosing the first $k$ means. In this notebook, we will choose randomly $k$ points from the input data set.

## Implementation 1/4 : utils and initialization

In [ ]:
#include <algorithm>
#include <cstdlib>
#include <functional>
#include <fstream>
#include <iostream>
#include <iterator>
#include <limits>
#include <numeric>
#include <string>
#include <vector>

The following functions are available in C++17 only, thus their implementation in this notebook.

In [ ]:
template <class InputIt1, class InputIt2, class T, class BinaryTransformOp, class BinaryReduceOp>
T transform_reduce(InputIt1 first1, InputIt1 last1, InputIt2 first2, T init, BinaryTransformOp tr_op, BinaryReduceOp red_op)
{
    T res = init;
    while (first1 != last1)
    {
        res = red_op(res, tr_op(*first1++, *first2++));
    }
    return res;
}

template <class InputIt1, class InputIt2, class T, class BinaryTransformOp>
T transform_reduce(InputIt1 first1, InputIt1 last1, InputIt2 first2, T init, BinaryTransformOp tr_op)
{
    return transform_reduce(first1, last1, first2, init, tr_op, std::plus<>());
}

template <class InputIt1, class InputIt2, class T>
T transform_reduce(InputIt1 first1, InputIt1 last1, InputIt2 first2, T init)
{
    return transform_reduce(first1, last1, first2, init, std::multiplies<>());
}

The following functions can be used for debugging your algorithm:

In [ ]:
template <class T>
void print(const std::vector<T>& data)
{
    std::cout << '{';
    std::copy(data.begin(), data.end(), std::ostream_iterator<T>(std::cout, ","));
    std::cout << "}\n";
}

template <class T>
void print(const std::vector<std::vector<T>>& data)
{
    std::for_each(data.begin(), data.end(), [](const auto& v) { print(v); });
}

We can now define the types and a function for initializing the data we will use in this notebook.

In [ ]:
using vector_type = std::vector<double>;
using matrix_type = std::vector<vector_type>;
using cluster_type = std::vector<size_t>;
using cluster_set = std::vector<cluster_type>;

In [ ]:
matrix_type init_data_set(size_t rows, size_t cols)
{
    matrix_type m (rows, vector_type(cols));
    std::for_each(m.begin(), m.end(), [](auto& v) {
        std::generate(v.begin(), v.end(), []() { return (rand() / (double)RAND_MAX); });
    });
    return m;
}

## Implementation 2/4 : assignment step

Write a function that computes the squared Euclidian distance between two observations.

In [ ]:
double squared_euclidian(const vector_type& lhs, const vector_type& rhs)

Write a function that given the cluster means and an observation, returns the cluster index for that observation.

In [ ]:
size_t compute_cluster_index(const vector_type& data, const matrix_type& means)

Write a function that assigns each observation to the cluster with the nearest distance.

In [ ]:
void assign_observations(const matrix_type& data, const matrix_type& means, cluster_set& cluster)

## Implementation 3/4 : update step

Write a function that computes the mean of a cluster.

In [ ]:
auto compute_cluster_mean(const cluster_type& cluster, const matrix_type& data)

Write a function that computes the means of all clusters.

In [ ]:
void compute_centroids(const cluster_set& clusters, const matrix_type& data, matrix_type& means)

## Implementation 4/4 : putting everything together

Implement the k-means algorithm thanks to the previously defined functions. It takes the data and the number of clusters as arguments, and returns a pair containing the cluster set and the centroids.

In [ ]:
matrix_type init_means(const matrix_type& data, size_t k)

In [ ]:
auto kmeans(const matrix_type& data, size_t k)

## Testing

In [ ]:
matrix_type data = init_data_set(15, 28);
auto res = kmeans(data, 3);
print(res.second)